<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import pandas as pd
import numpy as np
import random
import sys
import os

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/DS-Unit-4-Sprint-3-Deep-Learning/master/module1-rnn-and-lstm/sonnets.csv',
                 sep = '\n', header = None)
df.head()

,0
0,1
1,"From fairest creatures we desire increase,"
2,"That thereby beauty’s rose might never die,"
3,"But as the riper should by time decease,"
4,His tender heir might bear his memory:


In [3]:
df = df[~df[0].str.contains(r'\d')]

In [4]:
df.head()

,0
1,"From fairest creatures we desire increase,"
2,"That thereby beauty’s rose might never die,"
3,"But as the riper should by time decease,"
4,His tender heir might bear his memory:
5,"But thou contracted to thine own bright eyes,"


In [5]:
# Encode Data as Chars
text = " ".join(df[0].tolist())
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

# Resources and Stretch Goals

In [6]:
len(chars)

61

In [7]:
# Create the Sequence Data
# The lists are: encoded (chars encoded as integers)
#                sequences (40 chars per entry)
#                next_chars ()
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_chars = [] # One element for each sequence for prediction

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences', len(sequences))

sequences 18848


In [8]:
# Specify x & y - 3D matrix

# Initialize arrays with False
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [9]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #input shape of a singular observ.
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

W0209 12:46:14.043504 4383047104 deprecation.py:506] From /Users/JKMacBook/opt/anaconda3/envs/NN/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
def test(c_length):
    
    start_index = random.randint(0, len(text) - maxlen - 1)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(c_length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = int_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

char_length = 200

In [14]:
model.fit(x, y,
          batch_size=128,
          epochs=20)

Epoch 1/20
18848/18848 [==============================] - 25s 1ms/sample - loss: 2.8758
Epoch 2/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 2.6281
Epoch 3/20
18848/18848 [==============================] - 25s 1ms/sample - loss: 2.4650
Epoch 4/20
18848/18848 [==============================] - 25s 1ms/sample - loss: 2.3676
Epoch 5/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 2.2981
Epoch 6/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 2.2452
Epoch 7/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 2.1990
Epoch 8/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 2.1610
Epoch 9/20
18848/18848 [==============================] - 25s 1ms/sample - loss: 2.1254
Epoch 10/20
18848/18848 [==============================] - 26s 1ms/sample - loss: 2.0968
Epoch 11/20
18848/18848 [==============================] - 25s 1ms/sample - loss: 2.0700
Epoch 12/20
18848/18848 [=====

In [15]:
test(char_length)

----- Generating with seed: "an Time wastes life,   So thou prevent’s"
an Time wastes life,   So thou prevent’s thy sull thee boud the that here all the porty the furt,   Thou dy sendit stell sould for with the eaks price to dead the the than my deare thang thee the have thau headse of the the hing thy eell, a


## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN